<a href="https://colab.research.google.com/github/aditikhare16/CC-Practicals/blob/main/Practical%20-%206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np


In [5]:
from tensorflow.keras import datasets

# Load and normalize CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Check the shape
print(f'Training data shape: {x_train.shape}')
print(f'Testing data shape: {x_test.shape}')



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training data shape: (50000, 32, 32, 3)
Testing data shape: (10000, 32, 32, 3)


In [8]:
from tensorflow.keras import layers, models

def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model


In [9]:
import time
import tensorflow as tf

def benchmark(device):
    with tf.device(device):
        model = create_model()
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        start_time = time.time()
        history = model.fit(
            x_train, y_train,
            epochs=5,
            batch_size=64,
            verbose=2,
            validation_data=(x_test, y_test)
        )
        end_time = time.time()

        print(f"\nTraining on {device} took {end_time - start_time:.2f} seconds")
        return history


In [10]:
# Benchmark training on CPU
history_cpu = benchmark('/CPU:0')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
782/782 - 61s - 78ms/step - accuracy: 0.4602 - loss: 1.5047 - val_accuracy: 0.5396 - val_loss: 1.3315
Epoch 2/5
782/782 - 58s - 75ms/step - accuracy: 0.5941 - loss: 1.1580 - val_accuracy: 0.6211 - val_loss: 1.0913
Epoch 3/5
782/782 - 82s - 104ms/step - accuracy: 0.6437 - loss: 1.0230 - val_accuracy: 0.6303 - val_loss: 1.0593
Epoch 4/5
782/782 - 82s - 105ms/step - accuracy: 0.6716 - loss: 0.9463 - val_accuracy: 0.6607 - val_loss: 0.9608
Epoch 5/5
782/782 - 83s - 106ms/step - accuracy: 0.6907 - loss: 0.8913 - val_accuracy: 0.6755 - val_loss: 0.9454

Training on /CPU:0 took 390.88 seconds


In [11]:
# Set TensorFlow to use GPU for training
history_gpu = benchmark('/GPU:0')


Epoch 1/5
782/782 - 61s - 78ms/step - accuracy: 0.4602 - loss: 1.5056 - val_accuracy: 0.5463 - val_loss: 1.2621
Epoch 2/5
782/782 - 58s - 74ms/step - accuracy: 0.5951 - loss: 1.1555 - val_accuracy: 0.6075 - val_loss: 1.1319
Epoch 3/5
782/782 - 81s - 104ms/step - accuracy: 0.6397 - loss: 1.0350 - val_accuracy: 0.6470 - val_loss: 1.0214
Epoch 4/5
782/782 - 80s - 103ms/step - accuracy: 0.6689 - loss: 0.9542 - val_accuracy: 0.6729 - val_loss: 0.9633
Epoch 5/5
782/782 - 58s - 75ms/step - accuracy: 0.6917 - loss: 0.8899 - val_accuracy: 0.6724 - val_loss: 0.9406

Training on /GPU:0 took 365.00 seconds


In [12]:
!pip install tensorflow --quiet

In [14]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)



TensorFlow version: 2.18.0


In [15]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras import layers, models, datasets
import os

# Detect and initialize the TPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    tf.config.experimental_connect_to_cluster(resolver)  # Connect to the cluster
    tf.tpu.experimental.initialize_tpu_system(resolver)  # Initialize the TPU system

    strategy = tf.distribute.TPUStrategy(resolver)  # Create a TPU strategy
    print("TPU initialized successfully!")
except ValueError:
    print("TPU not found. Please enable TPU in Runtime settings.")

# Check for the TPU address
print("COLAB_TPU_ADDR:", os.environ.get("COLAB_TPU_ADDR"))


TPU not found. Please enable TPU in Runtime settings.
COLAB_TPU_ADDR: None
